In [1]:
!pip install gymnasium
!pip install stable_baselines3
!pip install sb3_contrib
!pip install gymnasium[mujoco]
!pip install stable-baselines[mpi]==2.10.0
!pip install swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [1]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import DDPG
import torch
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, BaseCallback
import time
from stable_baselines3.common.evaluation import evaluate_policy
from collections import defaultdict

In [2]:
os.chdir('/content/drive/MyDrive/Tilburg/Masters CSAI/Year 1 - Semester 2/Deep RL/Project/models/DDPG')

In [13]:
from stable_baselines3.common.evaluation import evaluate_policy
from collections import defaultdict

# # load and evaluate each model
# Code derived from provided assignment PDF and stable_baselines 3 documentation
seeds = [35]

results = defaultdict(list)
for seed in seeds:
  eval_env = make_vec_env('BipedalWalker-v3', n_envs = 4, seed = 1)
  #env = VecNormalize(env, norm_obs=True, norm_reward = True)
  eval_env = VecNormalize.load(f'DDPG_walker_seed_{seed}_24h.pkl',eval_env)
  eval_env.training = False  # Important to disable training mode when evaluating
  eval_env.norm_reward = True
  ev_model = DDPG.load(f'DDPG_walker_seed_{seed}_24h.zip')
  mean_reward, std_reward = evaluate_policy(ev_model, eval_env, n_eval_episodes=100, deterministic = True)
  print(f'seed : {seed}')
  print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')
  print('-----------')
  #results[m].append([eval_mass,mean_reward,std_reward])

seed : 12
mean_reward=-46.61 +/- 69.74408157934847
-----------
seed : 33
mean_reward=162.62 +/- 104.43631960638257
-----------


In [17]:
ls

DDPG_walker_seed_12_24h.pkl  DDPG_walker_seed_33_24h.pkl  DDPG_walker_seed_35_24h.pkl
DDPG_walker_seed_12_24h.zip  DDPG_walker_seed_33_24h.zip  DDPG_walker_seed_35_24h.zip


In [18]:
from stable_baselines3.common.evaluation import evaluate_policy
from collections import defaultdict

# # load and evaluate each model
# Code derived from provided assignment PDF and stable_baselines 3 documentation
seeds = [35]

results = defaultdict(list)
for seed in seeds:
  eval_env = make_vec_env('BipedalWalker-v3', n_envs = 4, seed = 1)
  #env = VecNormalize(env, norm_obs=True, norm_reward = True)
  eval_env = VecNormalize.load(f'DDPG_walker_seed_{seed}_24h.pkl',eval_env)
  eval_env.training = False  # Important to disable training mode when evaluating
  eval_env.norm_reward = True
  ev_model = DDPG.load(f'DDPG_walker_seed_{seed}_24h.zip')
  mean_reward, std_reward = evaluate_policy(ev_model, eval_env, n_eval_episodes=100, deterministic = True)
  print(f'seed : {seed}')
  print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')
  print('-----------')
  #results[m].append([eval_mass,mean_reward,std_reward])

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


seed : 35
mean_reward=-43.59 +/- 8.638806826067361
-----------


In [17]:
from stable_baselines3 import DDPG
import numpy as np
import time
seeds = [12, 33]
results = defaultdict(list)


for seed in seeds:
    eval_env = make_vec_env('BipedalWalker-v3', n_envs = 4, seed = 1)
    eval_env = VecNormalize.load(f'DDPG_walker_seed_{seed}_24h.pkl',eval_env)
    eval_env.training = False  # Disable training mode for evaluation
    eval_env.norm_reward = False

    model = DDPG.load(f'DDPG_walker_seed_{seed}_24h.zip')

    episode_rewards = []
    episode_lengths = []
    episode_times = []

    for _ in range(100):  # Evaluate for 100 episodes
        obs = eval_env.reset()
        done = False
        total_rewards = 0
        steps = 0
        start_time = time.time()
        while not done and steps < 1600:
          action, _ = model.predict(obs, deterministic=True)
          obs, reward, done, info = eval_env.step(action)
          done = done[0]
          #print(reward,done)
          total_rewards += reward
          steps += 1
        elapsed_time = time.time() - start_time
        episode_rewards.append(total_rewards)
        episode_lengths.append(steps)
        episode_times.append(elapsed_time)
        print(f'Seed {seed}: Episode reward: {total_rewards}, Steps: {steps}, Time : {elapsed_time}')

    mean_reward = sum(episode_rewards) / len(episode_rewards)
    mean_time = sum(episode_times) / len(episode_times)
    std_reward = (sum((x - mean_reward) ** 2 for x in episode_rewards) / len(episode_rewards)) ** 0.5
    std_time = (sum((x - mean_time) ** 2 for x in episode_times) / len(episode_times)) ** 0.5
    mean_steps = sum(episode_lengths) / len(episode_lengths)
    print(f'Seed: {seed}, Mean Reward: {np.mean(mean_reward)} +/- {np.mean(std_reward)}, Mean Steps: {mean_steps}, Mean time : {mean_time} +/- {std_time}')
    print('-----------')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


Seed 12: Episode reward: [ 70.8443   -92.33324  -30.110168  76.86531 ], Steps: 1600, Time : 2.67041015625
Seed 12: Episode reward: [-26.98044  -47.48285  -42.230503 -77.2688  ], Steps: 1600, Time : 2.6670758724212646
Seed 12: Episode reward: [-122.70879    59.508522 -125.357086  -74.50022 ], Steps: 1600, Time : 2.6952552795410156
Seed 12: Episode reward: [ 68.19784  -94.49552  -95.53837  -46.463795], Steps: 1600, Time : 3.5784761905670166
Seed 12: Episode reward: [ 66.465614    4.7116685 -81.96931    60.759567 ], Steps: 1600, Time : 2.9608500003814697
Seed 12: Episode reward: [-78.98175  -84.827705 -71.58748  -82.21988 ], Steps: 1600, Time : 2.690709114074707
Seed 12: Episode reward: [-83.52371  -62.380966  38.692528 -93.15801 ], Steps: 1364, Time : 2.292512893676758
Seed 12: Episode reward: [  79.87088   -66.207504  -91.599    -100.02121 ], Steps: 1600, Time : 2.655787467956543
Seed 12: Episode reward: [-135.92638   -43.43285  -121.22464    35.257557], Steps: 1600, Time : 3.8491635322